<a href="https://colab.research.google.com/github/arcsin42/data_visualization/blob/main/label_placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Клевцов Тимур, М05-013д  
2-SAT решатель для меток

In [ ]:
import math
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import networkx as nx
from itertools import *
#!pip install webcolors==1.3
#import webcolors
#from google.colab import drive
#drive.mount('/content/drive')

Чтобы считать из файла, изменить флаг from_file и прописать его имя в name

In [ ]:
from_file = False
name = 'test4.txt'

data = '''
20,20 10,10	0,0 0,10
20,25	10,10 10,0 0,10
25,20	10,10	0,0 10,0
25,25	10,10 10,0 10,10
40,30 10,10 10,10 0,0
38,20 10,5  10,5 10,0
45,26 5,5 5,5 5,0
'''
"""
Расположения прямоугольника равны координатам
правого верхнего угла относительно метки, то есть
pos_x,pos_y X_size,Y_size,  X_size,Y_szie 0,0 означает
положения справа сверху и слева снизу
Ось X вправо, ось Y вверх
Пересечения по границе допустимы
"""
L = 500 #размер рамки


class point():
  def __init__(self, x, y):
    self.x = x
    self.y = y
  def __add__(self, p):
    return point(self.x + p.x, self.y + p.y)
  def __sub__(self, p):
    return point(self.x - p.x, self.y - p.y)
  def __repr__(self):
    s = '({},{})'.format(self.x, self.y)
    return s
  def __abs__(self):
    return math.sqrt(self.x ** 2 + self.y ** 2)
  def parse(s):
    args = s.replace('.', ',').split(',')
    assert len(args) == 2, "Слишком мало аргументов у точки"
    return point(int(args[0]), int(args[1]))

class box():
  def __init__(self, s):
    arg = s.strip().replace('\t', ' ').split()
    args = [S for S in arg if len(S) > 0]
    self.pos = point.parse(args[0])
    self.size = point.parse(args[1])
    self.offsets = [point.parse(pos) for pos in args[2:4]]
    assert len(self.offsets) > 0, "Нету позиций у метки"
  def __repr__(self):
    s = 'Box pos={}, size={}, offsets={}\n'.format(self.pos, self.size, self.offsets)
    return s

def crossing(box1: box, box2: box, offset1, offset2):
  v2 = box1.pos + box1.offsets[offset1]
  v1 = v2 - box1.size
  w2 = box2.pos + box2.offsets[offset2]
  w1 = w2 - box2.size
  #print(v1, v2)
  #print(w1, w2)
  if w1.y >= v2.y or w1.x >= v2.x or w2.x <= v1.x or w2.y <= v1.y:
    return False

  return True

def border(): #создание мини квадратиков по периметру
  s = [str] * 4
  s[0] = '{},0  {},0  0,0 0,0'.format(L,L)
  s[1] = '{},{} 0,{}  0,0 0.0'.format(L,L,L)
  s[2] = '{},{} {},0  0,0 0.0'.format(L,L,L)
  s[3] = '0,{}  0,{}  0.0 0.0'.format(L,L)
  return [box(S) for S in s]

F = 0
if from_file:
  F = open(name, 'r')

if from_file:
  boxes = [box(line) for line in F.readlines()] + border()
else: 
  boxes = [box(line) for line in data.strip().split('\n')] + border()
#print(boxes)
"""
Нумерация клик алгоритмом Косараджу
"""
n = 2 * len(boxes)
g = [[] for i in range(n)] #здесь хранятся вершины a, not a, b, not b...
reverse_g = [[] for i in range(n)]  #обратный граф
used = [False] * n
order = []
comp = [-1] * n
answer = [int] * (n // 2) #будущие значения 0 или 1

def dfs(v):
  used[v] = True
  for child in g[v]:
    if not used[child]:
      dfs(child)
  order.append(v)
  return

def reverse_dfs(v, c):  #вершина и номер клики
  comp[v] = c
  for parent in reverse_g[v]:
    if comp[parent] == -1:
      reverse_dfs(parent, c)
  return

def SAT_solve():
  #used = [False] * n
  for i in range(n):  #прямой обход
    if not used[i]:
      dfs(i)
  j = 0
  for i in range(n):  #обратный обход
    v = order[n - 1 - i]
    #print(v, comp[v])
    if comp[v] == -1:
      reverse_dfs(v, j)
      j += 1

  for i in range(0, n, 2):
    if comp[i] == comp[i + 1]:  #если оказались в одной клике, все плохо
      print(boxes[i])
      return False
    if comp[i] > comp[i + 1]:
      answer[i // 2] = 0
    else:
      answer[i // 2] = 1
  return True


for i in range(n // 2): #создаем граф запретов
  for j in range(n // 2):
    if j <= i:
      continue
    for k, l in zip([0,0,1,1], [0,1,0,1]):
      if crossing(boxes[i], boxes[j], k, l):
        #print(i, j, k, l)
        g[2 * i + k].append(2 * j + 1 - l)
        g[2 * j + l].append(2 * i + 1 - k)

for i in range(n):  #создаем обратный граф
  for j in g[i]:
    reverse_g[j].append(i)


possible = SAT_solve() #решаем 2-SAT

print("Можно расположить:", possible)
if possible:
  print('Реализующее расположение:', answer[:-4])

#print(crossing(boxes[2], boxes[3], 1, 0))

Можно расположить: True
Реализующее расположение: [0, 1, 1, 1, 0, 1, 1]


In [ ]:
fig = go.Figure()
def random_color():
  colors = ['green', 'black', 'yellow', 'blue', 'grey', 'orange', 'red', 'darkblue']
  n = np.random.choice(len(colors))
  return colors[n]

_boxes = boxes[:-4]
if possible:
  for box in _boxes:
    i = _boxes.index(box)
    color = random_color()
    x = [0, -box.size.x, -box.size.x, 0, 0]
    y = [0, 0, -box.size.y, -box.size.y, 0]
    fig.add_trace(go.Scatter(
      x= [_x + box.offsets[answer[i]].x + box.pos.x for _x in x], 
      y= [_y + box.offsets[answer[i]].y + box.pos.y for _y in y], 
      line=dict(color=color, width=1)
      ))
    fig.add_trace(go.Scatter(
    x = [box.pos.x], 
    y = [box.pos.y], 
    mode = 'markers',
    marker=dict(size=[15], color=color),
    ))

m = [box.pos.x + box.size.x for box in _boxes]
max = 0
for l in m:
  if max < l:
    max = l
#задание характеристик холста
fig.update_layout(  
    autosize=False,
    width=max * 20,
    height=max * 20,
    margin=dict(l=10, r=10, b=10, t=10, pad=2),
    paper_bgcolor="LightSteelBlue",
    showlegend = False
)
fig.show()